In [3]:
import sys
import os

sys.path.append('/Users/raphaelravinet/Code')
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, select
from datetime import datetime, timedelta
import logging
import matplotlib.pyplot as plt
from dotenv import load_dotenv
import ta
from algo_trading.log_config import setup_logging
from Fin_Database.Data.connect import engine, DailyStockData, HourlyStockData, OneMinuteStockData, FiveMinuteStockData,FifteenMinuteStockData, StockSplits, StockNews, CompanyFinancials
from algo_trading.Pre_Processing.pre_processing import PreProcessing
from algo_trading.Feature_Engineering.feature_engineering import TechnicalIndicators

In [4]:
aapl_min = pd.read_parquet('/Users/raphaelravinet/Code/algo_trading/Datasets/Minute_Data/aapl_minute.parquet')
msft_min = pd.read_parquet('/Users/raphaelravinet/Code/algo_trading/Datasets/Minute_Data/msft_minute.parquet')
aapl_hourly = pd.read_parquet('/Users/raphaelravinet/Code/algo_trading/Datasets/Hourly_Data/aapl_hourly.parquet')
msft_hourly = pd.read_parquet('/Users/raphaelravinet/Code/algo_trading/Datasets/Hourly_Data/msft_hourly.parquet')
aapl_daily = pd.read_csv('/Users/raphaelravinet/Code/algo_trading/Datasets/Daily_Data/aapl_daily.csv')
msft_daily = pd.read_csv('/Users/raphaelravinet/Code/algo_trading/Datasets/Daily_Data/msft_daily.csv')


In [5]:
aapl_daily = PreProcessing(aapl_daily).setting_index().df
msft_daily = PreProcessing(msft_daily).setting_index().df
aapl_min = PreProcessing(aapl_min).filter_market_hours().setting_index().df
msft_min = PreProcessing(msft_min).filter_market_hours().setting_index().df
aapl_hourly = PreProcessing(aapl_hourly).filter_market_hours().setting_index().df
msft_hourly = PreProcessing(msft_hourly).filter_market_hours().setting_index().df



#### Applying the Approach from Young Li's Paper: 
**"A Practical Model for Prediction of Intraday Volatility"**

**Potential Improvement:**

1. **Change the daily vol model from EWMA to a GARCH-type model**


In [5]:
class IntraBarsProfile:
    def __init__(self, hf_df, lf_df, daily_df, timespan):
        """args:
        hf_df: higher frequency dataframe
        lf_df: lower frequency dataframe
        daily_df: daily dataframe
        timespan: timespan of the lower frequency data to aggregate : '15M', '1H', '1D', '1W....'"""
        self.hf_df = hf_df
        self.lf_df = lf_df
        self.daily_df = daily_df
        self.timespan = timespan
        
        
    def hurst(self):
        pass

In [6]:
class IntradayVol(IntraBarsProfile):
    def __init__(self, hf_df, lf_df, daily_df, timespan, decay_factor = 0.94):
        super().__init__(hf_df, lf_df, daily_df, timespan)
        self.decay_factor = decay_factor

    
    def calculate_daily_vol(self):
        """ Calculate daily vol using EWMA model """
        self.daily_df['daily_vol_squared'] = np.zeros(len(self.daily_df), dtype=float)
        
        self.daily_df['squared_log_ret'] = self.daily_df['log_ret'] ** 2
        self.daily_df['squared_log_ret'].bfill(inplace=True)

        for i in range(1, len(self.daily_df)):
            self.daily_df.loc[self.daily_df.index[i], 'daily_vol_squared'] = (
                self.decay_factor * self.daily_df['daily_vol_squared'].iloc[i-1] +
                (1 - self.decay_factor) * self.daily_df['squared_log_ret'].iloc[i]
            )
        
        return np.sqrt(self.daily_df['daily_vol_squared'])


    
    def calculate_diurnal_profile(self):
        """Calculate diurnal profile. Basically the average of the Garman-Klass vol for each time of the day. 
        It is calculated using the aggregated data of the higher frequency data"""
        aggregated_df = self.hf_df.groupby(self.hf_df.index.floor(self.timespan)).aggregate({'open': 'first', 
                                                               'high': 'max', 
                                                               'low': 'min', 
                                                               'close': 'last'})
        garma_klass_vol = np.sqrt(0.5*(np.log(aggregated_df['high']) - np.log(aggregated_df['low']))**2 - (2 * np.log(2) -1) * (np.log(aggregated_df['close']) - np.log(aggregated_df['open']))**2)
        q25 = garma_klass_vol.groupby(aggregated_df.index.time).quantile(0.25)
        q50 = garma_klass_vol.groupby(aggregated_df.index.time).quantile(0.5)
        q75 = garma_klass_vol.groupby(aggregated_df.index.time).quantile(0.75)
        
        diurnal_profile = (q25 + q50 + q75) / 3
        diurnal_profile = diurnal_profile / diurnal_profile.mean()
        
        return diurnal_profile
    
    
    def predict_vol(self):
        """ Predict vol for the next time interval"""
        daily_vol = self.calculate_daily_vol().shift(1)
        diurnal_profile = self.calculate_diurnal_profile()
        avg_diurnal_profile = diurnal_profile.mean()
        
        # time-scaling factor (rho_t)
        time_scaling_factor = 1  

        self.lf_df['vol_forecasts'] = np.nan
        for i in range(len(self.lf_df)):
            current_time = self.lf_df.index[i].time()
            current_date = self.lf_df.index[i].date()
            
            daily_vol_value = daily_vol.loc[pd.Timestamp(current_date)]
            if current_time in diurnal_profile.index:
                self.lf_df.loc[self.lf_df.index[i], 'vol_forecasts'] = (
                    daily_vol_value * time_scaling_factor * (diurnal_profile[current_time] / avg_diurnal_profile)
                )
            else:
                self.lf_df.loc[self.lf_df.index[i], 'vol_forecasts'] = np.nan
        return self.lf_df


    
        

In [7]:
fast_periods = [5, 10, 30]
slow_periods = [50, 100, 200,300]
periods = fast_periods + slow_periods

In [8]:
aapl_daily = TechnicalIndicators(aapl_daily).add_technical_indicators(periods)
msft_daily = TechnicalIndicators(msft_daily).add_technical_indicators(periods)
aapl_hourly = TechnicalIndicators(aapl_hourly).add_technical_indicators(periods)
msft_hourly = TechnicalIndicators(msft_hourly).add_technical_indicators(periods)
aapl_min = TechnicalIndicators(aapl_min).add_technical_indicators(periods)
msft_min = TechnicalIndicators(msft_min).add_technical_indicators(periods)


In [ ]:
timespan = '1H' 
aapl_vol_model = IntradayVol(hf_df=aapl_min, lf_df=aapl_hourly, daily_df=aapl_daily, timespan=timespan)
msft_vol_model = IntradayVol(hf_df=msft_min, lf_df=msft_hourly, daily_df=msft_daily, timespan=timespan)

In [ ]:
aapl_predicted_vol

In [ ]:
# AAPL Volatility Calculations
aapl_daily_vol = aapl_vol_model.calculate_daily_vol()
aapl_diurnal_profile = aapl_vol_model.calculate_diurnal_profile()
aapl_predicted_vol = aapl_vol_model.predict_vol()

# MSFT Volatility Calculations
msft_daily_vol = msft_vol_model.calculate_daily_vol()
msft_diurnal_profile = msft_vol_model.calculate_diurnal_profile()
msft_predicted_vol = msft_vol_model.predict_vol()


In [ ]:
#Using the Garma Klass vol formula as target
aapl_predicted_vol['target'] = (np.sqrt(0.5 * (np.log(aapl_predicted_vol['high']) - np.log(aapl_predicted_vol['low']))**2 -
                              (2 * np.log(2) - 1) * (np.log(aapl_predicted_vol['close']) - np.log(aapl_predicted_vol['open']))**2)).shift(-1)

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(aapl_predicted_vol['vol_forecasts'], label='Forecasted Volatility')
plt.plot(aapl_predicted_vol['target'], label='Target Volatility')
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(aapl_predicted_vol['vol_forecasts'], label='Forecasted Volatility')
plt.plot(np.abs(aapl_predicted_vol['log_ret']), label='Abs Log Ret')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import root_mean_squared_error as rmse

In [ ]:
aapl_predicted_vol2 = aapl_predicted_vol.iloc[15:-1].copy()

In [ ]:
rmse(aapl_predicted_vol2['vol_forecasts'], aapl_predicted_vol2['target'])